In [ ]:
# from rw_ve_plot import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PythonMeta as PMA
%load_ext autoreload
%autoreload 2
plt.style.use(r"./RW_visualization.mplstyle")


In [2]:
# import warnings filter
# from pandas.core.common import SettingWithCopyWarning
import warnings
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
pd.options.mode.chained_assignment = None 

In [3]:
# Run vaccine_efficacy_CIs.ipynb to update the data
# df1 = pd.read_excel(r'C:\Users\dipes\python\jupyter_notebook_files\Vaccine details.xlsx')
vaccine_detail = pd.read_excel('rw_vaccine_details.xlsx')
# vaccine_detail = pd.read_excel('./Vaccine/RW_vaccine_details.xlsx')
vaccine_detail.columns = vaccine_detail.columns.str.strip().str.lower().str.replace(' ', '_').\
    str.replace('(', '').str.replace(')', '')

# Remove all vaccine
vaccine_detail = vaccine_detail[vaccine_detail.vaccine!='All vaccine']

In [4]:
# Replace the vaccine efficacy and Ci by my estimations
vaccine_detail['efficacy_in_%'] = vaccine_detail['rw_efficacy_in_%']
vaccine_detail['lower'] = vaccine_detail['rw_lower']
vaccine_detail['upper'] = vaccine_detail['rw_upper']

In [9]:
# Drop empty vaccine efficacy
df = vaccine_detail[~np.isnan(vaccine_detail['efficacy_in_%'])]

In [ ]:
df

In [15]:
meta_df = pd.DataFrame({
        'study': df['reference'],  # Study identifier
        'vax_n': df['no_of_participants_in_vaccine_group'],  # Vaccine group size
        'vax_cases': df['no_of_infected_in_vaccine_group'],  # Cases in vaccine group
        'ctrl_n': df['no_of_participants_in_placebo_groupcontrol_group'],  # Control group size
        'ctrl_cases': df['no_of_infected_in_placebo_group'],  # Cases in control group
        'vaccine': df['vaccine'],  # Vaccine type
        'variant': df['variant'],  # Virus variant
        'ave': df['ave']  # Type of vaccine efficacy measured
    })

In [ ]:
meta_df

In [17]:
# Convert numeric columns to appropriate type
numeric_cols = ['vax_n', 'vax_cases', 'ctrl_n', 'ctrl_cases']
for col in numeric_cols:
    meta_df[col] = pd.to_numeric(meta_df[col], errors='coerce')

In [ ]:
meta_df

In [27]:
def showstudies(studies,dtype):    
    #show continuous data
    if dtype.upper()=="CONT":
        text = "%-10s %-30s %-30s \n"%("Study ID","Experiment Group","Control Group")
        text += "%-10s %-10s %-10s %-10s %-10s %-10s %-10s \n"%(" ","m1","sd1","n1","m2","sd2","n2")
        for i in range(len(studies)):
            text += "%-10s %-10s %-10s %-10s %-10s  %-10s %-10s \n"%(
            studies[i][6],        #study ID
            str(studies[i][0]),   #mean of group1
            str(studies[i][1]),   #SD of group1
            str(studies[i][2]),   #total num of group1
            str(studies[i][3]),   #mean of group2
            str(studies[i][4]),   #SD of group2
            str(studies[i][5])    #total num of group2
            )
        return text
        
    #show dichotomous data
    text = "%-10s %-20s %-20s \n"%("Study ID","Experiment Group","Control Group")
    text += "%-10s %-10s %-10s %-10s %-10s \n"%(" ","e1","n1","e2","n2")
    for i in range(len(studies)):
        text += "%-10s %-10s %-10s %-10s %-10s \n"%(
        studies[i][4],        #study ID
        str(studies[i][0]),   #event num of group1
        str(studies[i][1]),   #total num of group1
        str(studies[i][2]),   #event num of group2
        str(studies[i][3])    #total num of group2
        )
    return text

def showresults(rults):
    text = "%-10s %-6s  %-18s %-10s"%("Study ID","n","ES[95% CI]","Weight(%)\n")    
    for i in range(1,len(rults)):
        text += "%-10s %-6d  %-4.2f[%.2f %.2f]   %6.2f\n"%(   # for each study
        rults[i][0],     #study ID
        rults[i][5],     #total num
        rults[i][1],     #effect size
        rults[i][3],     #lower of CI
        rults[i][4],     #higher of CI
        100*(rults[i][2]/rults[0][2])  #weight
        )
    text += "%-10s %-6d  %-4.2f[%.2f %.2f]   %6d\n"%(         # for total effect
        rults[0][0],     #total effect size name
        rults[0][5],     #total N (all studies)
        rults[0][1],     #total effect size
        rults[0][3],     #total lower CI
        rults[0][4],     #total higher CI
        100
        )  
    text += "%d studies included (N=%d)\n"%(len(rults)-1,rults[0][5])
    text += "Heterogeneity: Tau\u00b2=%.3f "%(rults[0][12]) if not rults[0][12]==None else "Heterogeneity: "
    text += "Q(Chisquare)=%.2f(p=%s); I\u00b2=%s\n"%(
        rults[0][7],     #Q test value
        rults[0][8],     #p value for Q test
        str(round(rults[0][9],2))+"%")   #I-square value
    text += "Overall effect test: z=%.2f, p=%s\n"%(rults[0][10],rults[0][11])  #z-test value and p-value
    
    return text

In [ ]:
import pandas as pd
import numpy as np
import PythonMeta as PMA

def prepare_data_for_pythonmeta(excel_file, ave, variant='SARS-CoV-2'):
    # Read the Excel file
    df = pd.read_excel(excel_file)
    
    # Replace 'X' with NaN
    df = df.replace('X', np.nan)
    
    df = df[df['ave']==ave]
    df = df[df['variant']==variant]

    # Convert numeric columns to appropriate type
    numeric_cols = ['no_of_participants_in_vaccine_group', 'no_of_infected_in_vaccine_group',
                   'no_of_participants_in_placebo_groupcontrol_group', 'no_of_infected_in_placebo_group']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Create formatted strings for PythonMeta
    study_strings = []
    for _, row in df.iterrows():
        if pd.notna(row['no_of_infected_in_vaccine_group']) and pd.notna(row['no_of_infected_in_placebo_group']):
            study_string = f"{row['reference']}, {int(row['no_of_infected_in_vaccine_group'])}, {int(row['no_of_participants_in_vaccine_group'])}, {int(row['no_of_infected_in_placebo_group'])}, {int(row['no_of_participants_in_placebo_groupcontrol_group'])}"
            study_strings.append(study_string)
    
    return study_strings

# Prepare the data
study_data = prepare_data_for_pythonmeta('rw_vaccine_details.xlsx', ave='Symptomatic')

# Set up the analysis settings
settings = {
    "datatype": "CATE",  # for CATEgorical/binary data
    "models": "Random",  # Random effects model
    "algorithm": "MH",   # Mantel-Haenszel method
    "effect": "RR"      # Risk Ratio as effect measure
}

# Run the meta-analysis
def run_meta_analysis(study_data, settings):
    d = PMA.Data()  # Load Data class
    m = PMA.Meta()  # Load Meta class
    f = PMA.Fig()   # Load Fig class
    
    # Set data type
    d.datatype = settings["datatype"]
    
    # Load data
    studies = d.getdata(study_data)
    print("Studies included:")
    print(showstudies(studies, d.datatype))
    
    # Set meta-analysis parameters
    m.datatype = d.datatype
    m.models = settings["models"]
    m.algorithm = settings["algorithm"]
    m.effect = settings["effect"]
    
    # Perform the analysis
    results = m.meta(studies)
    
    # Show results
    print(f"\n{m.models} {m.algorithm} {m.effect} Meta-Analysis Results:")
    print(showresults(results))
    
    # Generate plots
    f.forest(results).show()
    f.funnel(results).show()
    
    # Perform Egger's test
    print("\nEgger's test results:")
    print(m.Eggers_test(results))

# Run the analysis
run_meta_analysis(study_data, settings)